# Human Pose Estimation sec - 1.1

This is a Notebook coded on Google Colab

* This notebook is used to:
      - Load Custom Created functions
      - Load pre-defined model architecture
      - Load data from pre-defined pipelines
      - Train the model with 50 epochs on colab with GPU
      - A callback is defined to save the best model

In [ ]:
#Moving the data files from drive to current runtime, inorder to solve any bottleneck problems
# !cp drive/My\ Drive/Colab\ Notebooks/File.zip  file.zip
# !unzip file.zip
# !cp ../model.h5 ../content/drive/My Drive/Colab Notebooks

In [2]:
#Change the working directory
import os
os.chdir('Custom_Functions/')
#Import all necessary libraries
import pandas as pd, numpy as np, tensorflow
from tensorflow.keras.losses import MSE as mse
from tensorflow.keras.callbacks import ModelCheckpoint as mc
#Importint the custome functions that returns the data
from Data_prep import prep
from Model_Generator import run
from Metrics import coeff_determination, optimizer

Using TensorFlow backend.


In [ ]:
#Loading the action_joints dataset only to use the df.shape as output shape for our model
df = pd.read_csv('../Data/Action_Dataset/action_joints.csv')
df.columns= list(range(df.shape[1]))

Train, Validation, Test = prep() #Returns data pipelines from the custom function
model=run() #Returns the untrained model architecture

In [ ]:
adam=optimizer() #loads cutomized adam optimizer from the function
metrics=[coeff_determination] #using the custom R2 metric
model.compile(optimizer= optimizer, loss=mse,  metrics=metrics)
#Creating a callback to help save 'best model only'
mcp=mc('../Custom_Models/Keras_Models_H5/final_model.h5', monitor='val_coeff_determination', patience=10, save_best_only=True, 
       mode= 'max', baseline=None, restore_best_weights=True)
#The best model is automatically saved at the given location

'''Fit the model with data to enable training.. on Colab with GPU, this takes 35 minutes to run 50 epochs
and return train r2 of 96.32 and val r2 of 92.13'''
model.fit(Train, validation_data=Validation, epochs=50, callbacks=[mcp], 
          steps_per_epoch= Train.samples//batch_size,
          validation_steps= Validation.samples//df.shape[0])

# The Final Keypoints predition model can be loaded to run predictions directly or linked to a WebApplication or API

* I further used this model to predict keypoints and serve as input to my [Hello/Namaste] classification model in sec 1.4 where I try to predict Hello/Namaste from live webcam feed.

In [3]:
from Model_Generator import run2
from Data_prep import prep2
import tensorflow
from tensorflow.keras import regularizers

In [4]:
model=run2()
X_Train, Y_Train, X_Val, Y_Val, transformer = prep2()

Model is saved to '/Untrained_Classification_Model.h5'


In [8]:
%%time
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 
mcp=mc('../Custom_Models/Keras_Model_H5/final_classification_model.h5',
       monitor='val_loss', patience=20, save_best_only=True,
       mode= 'min', restore_best_weights=True)
'''Fit the model with data to enable training.. on local cpu runtime [core i5, 8gb ram, 8gb graphics],
this takes 4 seconds to run 50 epochs and return train acc of 99.7 and val acc of 99.7, which is considered 100%'''
model.fit(X_Train, Y_Train, validation_data =(X_Val, Y_Val), epochs=50, callbacks= [mcp])

Epoch 1/50
1/1 [==============================] - 0s 207ms/step - loss: 0.0145 - accuracy: 1.0000 - val_loss: 0.5271 - val_accuracy: 0.6667
Epoch 2/50
1/1 [==============================] - 0s 60ms/step - loss: 0.0171 - accuracy: 1.0000 - val_loss: 0.5344 - val_accuracy: 0.6667
Epoch 3/50
1/1 [==============================] - 0s 99ms/step - loss: 0.0150 - accuracy: 1.0000 - val_loss: 0.2427 - val_accuracy: 1.0000
Epoch 4/50
1/1 [==============================] - 0s 82ms/step - loss: 0.0137 - accuracy: 1.0000 - val_loss: 0.1478 - val_accuracy: 1.0000
Epoch 5/50
1/1 [==============================] - 0s 82ms/step - loss: 0.0138 - accuracy: 1.0000 - val_loss: 0.1126 - val_accuracy: 1.0000
Epoch 6/50
1/1 [==============================] - 0s 45ms/step - loss: 0.0135 - accuracy: 1.0000 - val_loss: 0.1373 - val_accuracy: 1.0000
Epoch 7/50
1/1 [==============================] - 0s 43ms/step - loss: 0.0131 - accuracy: 1.0000 - val_loss: 0.3052 - val_accuracy: 1.0000
Epoch 8/50
1/1 [==========